# Parsing Randomness
This is an example showing the principles of "Free Generators", a data structure both interpretable as:
- A Generator
- A Parser

First, like in the paper [TODO], the "original" Parser and Generator will be shown. Next, they will both be build using the introduced data structure `FreeGen`.

The utilised example are Binary Boolean trees, first introduced here:

## Binary Boolean trees
The trees will have the following elements:
- `N` stands for a node
  - A Node has 2 children (other tree)
  - and a Boolean value assigned (`0` or `1`)
- `L` stands for leaf

An example could look like this:
```haskell
     N: 0 
   /     \
  L      N: 1
       /     \
      N: 0    L
    /     \
   L       L
```

A **Generator** creates such a data structure at random. A **Parser** gets some String input (for the shown example: `N 0 L N 1 N 0 L L L`) and creates the according instance of the data structure.


## Implementation of original Generator and Parser
To show the analogy of the structure in Generators and Parsers, this section will start with the original implementation of both for creating (either making or parsing) a Binary Boolean Tree.

Both the utilised Generator and Parser are already existing implementations from other libraries.

In [ ]:
import Test.QuickCheck (Gen)
import qualified Test.QuickCheck as Gen

generateTree :: Int -> Gen Tree
generateTree 0 = return Leaf
generateTree h = do
  c <- frequency [(1,False), (3,True)]
  case c of
    False -> return Leaf
    True -> do
      x <- genBool
      l <- genTree (h-1)
      r <- genTree (h-1)
      return (Node l x r)